In [ ]:
import sys
# setting path
sys.path.append('../')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
RESULTS_PATH = MAIN_PATH + "results/metrics/"
HISTORY_PATH = MAIN_PATH + "history/"
MODEL_NAME = "mobilenetv3s_0pe_224ps_0.3theta_0.05sigmaRui_60000nsamples"

SAVE_PATH = MAIN_PATH + "results/figures/"
history_file = HISTORY_PATH + MODEL_NAME + ".csv"

In [ ]:
path = RESULTS_PATH + "results_camelyon.csv"
df = pd.read_csv(path)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
df = df.drop(columns=["pretraining_epochs", "cnet_name", "mnet_name", "patch_size", "n_samples"])
df = df.drop(columns=[c for c in df.columns if ("loss" in c) or ("psnr" in c) or ("ssim" in c)])
df["theta_val"] = df["theta_val"].astype(str)
avg_df = df.groupby(["sigmaRui_sq", "theta_val"]).mean().reset_index()
std_df = df.groupby(["sigmaRui_sq", "theta_val"]).std().reset_index()
#avg_df["theta_val"] = avg_df["theta_val"].astype(float)
#std_df["theta_val"] = std_df["theta_val"].astype(float)
avg_df = avg_df.sort_values(by=["sigmaRui_sq", "theta_val"])
std_df = std_df.sort_values(by=["sigmaRui_sq", "theta_val"])
new_df = avg_df.join(std_df, lsuffix='_avg', rsuffix='_std')
new_df = new_df.drop(columns=["theta_val_std"])
new_df = new_df.rename(columns={"theta_val_avg": "theta_val"})
new_df = new_df.drop(columns=["sigmaRui_sq_std"])
new_df = new_df.rename(columns={"sigmaRui_sq_avg": "sigmaRui_sq"})
new_df = new_df.sort_values(by=["sigmaRui_sq", "theta_val"])
new_df

In [ ]:
plt.rcParams.update({'font.family' : 'sans-serif', 'font.size': 16})
FIGSIZE = (8,6)

In [ ]:
metric_name = "camelyon_test_mse_rec"
fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    #ax.plot(df["theta_val"], df["camelyon_test_mse_rec_avg"], 'o-', label=r"$\sigma^2={}$".format(sigma), color=color_vec[i])
    ax.errorbar(df["theta_val"], df[f"{metric_name}_avg"], yerr=df[f"{metric_name}_std"], lw=2, elinewidth=2, capsize=4, capthick=2, fmt='s-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]))
ax.legend()
ax.set_ylim(0.0, 0.003)
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("Reconstruction MSE")
plt.show()

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    #ax.plot(df["theta_val"], df["camelyon_test_mse_rec_avg"], 'o-', label=r"$\sigma^2={}$".format(sigma), color=color_vec[i])
    ax.errorbar(df["theta_val"], df[f"{metric_name}_avg"], yerr=df[f"{metric_name}_std"], lw=2, elinewidth=2, capsize=4, capthick=2, fmt='s-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]))
    ax.fill_between(df["theta_val"], df[f"{metric_name}_avg"] - df[f"{metric_name}_std"], df[f"{metric_name}_avg"] + df[f"{metric_name}_std"], alpha=0.2)
ax.legend()
ax.set_ylim(0.0, 0.003)
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("Reconstruction MSE")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    ax.plot(df["theta_val"], df[f"{metric_name}_avg"], 's-', lw=2, label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i])
ax.legend()
ax.set_ylim(0.0, 0.003)
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("Reconstruction MSE")
plt.show()

In [ ]:
metric_name = "camelyon_test_mse_rec"

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
width = 0.2
ind = np.arange(len(new_df["theta_val"].unique()))
for i in range(len(color_vec)):
    df = new_df[new_df["sigmaRui_sq"] == sigma_vec[i]]
    ax.bar(x = ind + i*width, height = df[f"{metric_name}_avg"], width = width, label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i], edgecolor='black', yerr=df[f"{metric_name}_std"], capsize=3)
ax.set_xticks(ind+(3*width)/2, new_df["theta_val"].unique())
ax.legend()
ax.set_ylim(0.0, 0.003)
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("Reconstruction MSE")
plt.savefig(SAVE_PATH + "mse_cam_bar.pdf", bbox_inches='tight')
plt.show()

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(df[df['mnet_name'] != 'mobilenetv3s'].index)
df = df.drop(df[df['pretraining_epochs'] != 1.0].index)
df = df[ df["cnet_name"] == "unet6" ]
df = df[ (df["theta_val"] == 0.0) | (df["theta_val"] == 0.3) | (df["theta_val"] == 0.5) | (df["theta_val"] == 0.7) | (df["theta_val"] == 1.0) ]
df = df.drop(columns=["pretraining_epochs", "cnet_name", "mnet_name", "patch_size", "n_samples"])
df = df.drop(columns=[c for c in df.columns if ("loss" in c)])

df = df[[ 'sigmaRui_sq', 'theta_val',
    'wssb_Lung_test_mse_rec', 'wssb_Breast_test_mse_rec', 'wssb_Colon_test_mse_rec',
    'wssb_Lung_test_psnr_gt_h', 'wssb_Lung_test_psnr_gt_e',
    'wssb_Breast_test_psnr_gt_h', 'wssb_Breast_test_psnr_gt_e',
    'wssb_Colon_test_psnr_gt_h', 'wssb_Colon_test_psnr_gt_e',
    'wssb_Lung_test_ssim_gt_h', 'wssb_Lung_test_ssim_gt_e',
    'wssb_Breast_test_ssim_gt_h', 'wssb_Breast_test_ssim_gt_e',
    'wssb_Colon_test_ssim_gt_h', 'wssb_Colon_test_ssim_gt_e'
]]
df = df.sort_values(by=["theta_val"])
df["theta_val"] = df["theta_val"].astype(str)
#df = df[[c for c in df.columns if "psnr_gt" in c]]
# df = df[["Name"] + list(df.columns[:-1])]
# df = df.append({"Name" : "BCDnet (old)", 
#     "wssb_Lung_test_psnr_gt_h" : 27.06, "wssb_Lung_test_psnr_gt_e" : 25.69, 
#     "wssb_Breast_test_psnr_gt_h" : 21.81, "wssb_Breast_test_psnr_gt_e" : 24.53, 
#     "wssb_Colon_test_psnr_gt_h" : 20.32, "wssb_Colon_test_psnr_gt_e" : 21.70}, ignore_index=True)
# df = df.append({"Name" : "BKSVD", 
#     "wssb_Lung_test_psnr_gt_h" : 32.67, "wssb_Lung_test_psnr_gt_e" : 30.61, 
#     "wssb_Breast_test_psnr_gt_h" : 32.20, "wssb_Breast_test_psnr_gt_e" : 29.43, 
#     "wssb_Colon_test_psnr_gt_h" : 34.08, "wssb_Colon_test_psnr_gt_e" : 33.32}, ignore_index=True)

df['wssb_mean_test_mse_rec'] = df[[c for c in df.columns if "mse_rec" in c]].mean(axis=1)

df["wssb_mean_test_psnr_gt_h"] = df[[c for c in df.columns if "psnr_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt_e"] = df[[c for c in df.columns if "psnr_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt"] = df[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]].mean(axis=1)

df["wssb_mean_test_ssim_gt_h"] = df[[c for c in df.columns if "ssim_gt_h" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt_e"] = df[[c for c in df.columns if "ssim_gt_e" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt"] = df[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]].mean(axis=1)


df = df[['sigmaRui_sq', 'theta_val', 'wssb_mean_test_mse_rec', 'wssb_mean_test_psnr_gt', 'wssb_mean_test_ssim_gt']]

avg_df = df.groupby(["sigmaRui_sq", "theta_val"]).mean().reset_index()
std_df = df.groupby(["sigmaRui_sq", "theta_val"]).std().reset_index()
#avg_df["theta_val"] = avg_df["theta_val"].astype(float)
#std_df["theta_val"] = std_df["theta_val"].astype(float)
avg_df = avg_df.sort_values(by=["sigmaRui_sq", "theta_val"])
std_df = std_df.sort_values(by=["sigmaRui_sq", "theta_val"])
new_df = avg_df.join(std_df, lsuffix='_avg', rsuffix='_std')
new_df = new_df.drop(columns=["theta_val_std"])
new_df = new_df.rename(columns={"theta_val_avg": "theta_val"})
new_df = new_df.drop(columns=["sigmaRui_sq_std"])
new_df = new_df.rename(columns={"sigmaRui_sq_avg": "sigmaRui_sq"})
new_df = new_df.sort_values(by=["sigmaRui_sq", "theta_val"])
new_df

In [ ]:
metric_name = "wssb_mean_test_mse_rec"

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
width = 0.2
ind = np.arange(len(new_df["theta_val"].unique()))
for i in range(len(color_vec)):
    df = new_df[new_df["sigmaRui_sq"] == sigma_vec[i]]
    ax.bar(x = ind + i*width, height = df[f"{metric_name}_avg"], width = width, label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i], edgecolor='black', yerr=df[f"{metric_name}_std"], capsize=3)
ax.set_xticks(ind+(3*width)/2, new_df["theta_val"].unique())
ax.legend()
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("Reconstruction MSE")
#ax.set_ylim(19, 27)
plt.savefig(SAVE_PATH + "mse_wssb_bar.pdf", bbox_inches='tight')
plt.show()

In [ ]:
metric_name = "wssb_mean_test_psnr_gt"

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    #ax.plot(df["theta_val"], df["camelyon_test_mse_rec_avg"], 'o-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i])
    ax.errorbar(df["theta_val"], df[f"{metric_name}_avg"], yerr=df[f"{metric_name}_std"], lw=2, elinewidth=2, capsize=4, capthick=2, fmt='s-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]))
ax.legend()
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("PSNR")
ax.set_ylim(19, 25)
plt.show()

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    #ax.plot(df["theta_val"], df["camelyon_test_mse_rec_avg"], 'o-', label=r"$\sigma^2={}$".format(sigma), color=color_vec[i])
    ax.errorbar(df["theta_val"], df[f"{metric_name}_avg"], yerr=df[f"{metric_name}_std"], lw=2, elinewidth=2, capsize=4, capthick=2, fmt='s-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]))
    ax.fill_between(df["theta_val"], df[f"{metric_name}_avg"] - df[f"{metric_name}_std"], df[f"{metric_name}_avg"] + df[f"{metric_name}_std"], alpha=0.2)
ax.legend()
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("PSNR")
ax.set_ylim(19, 25)

plt.show()

In [ ]:
metric_name = "wssb_mean_test_psnr_gt"
fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    ax.plot(df["theta_val"], df[f"{metric_name}_avg"], 's-', lw=2, label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i])
ax.legend()
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("PSNR")
ax.set_ylim(19, 25)
plt.show()

In [ ]:
metric_name = "wssb_mean_test_psnr_gt"

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
width = 0.2
ind = np.arange(len(new_df["theta_val"].unique()))
for i in range(len(color_vec)):
    df = new_df[new_df["sigmaRui_sq"] == sigma_vec[i]]
    ax.bar(x = ind + i*width, height = df[f"{metric_name}_avg"], width = width, label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i], edgecolor='black', yerr=df[f"{metric_name}_std"], capsize=3)
ax.set_xticks(ind+(3*width)/2, new_df["theta_val"].unique())
ax.legend()
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("PSNR")
ax.set_ylim(19, 27)
plt.savefig(SAVE_PATH + "psnr_bar.pdf", bbox_inches='tight')
plt.show()


In [ ]:
FIGSIZE = (8,6)
metric_name = "wssb_mean_test_ssim_gt"

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    #ax.plot(df["theta_val"], df["camelyon_test_mse_rec_avg"], 'o-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i])
    ax.errorbar(df["theta_val"], df[f"{metric_name}_avg"], yerr=df[f"{metric_name}_std"], lw=2, elinewidth=2, capsize=4, capthick=2, fmt='s-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]))
ax.legend()
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("PSNR (dB)")
ax.set_ylim(0.65, 0.9)
plt.show()

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    #ax.plot(df["theta_val"], df["camelyon_test_mse_rec_avg"], 'o-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i])
    ax.errorbar(df["theta_val"], df[f"{metric_name}_avg"], yerr=df[f"{metric_name}_std"], lw=2, elinewidth=2, capsize=4, capthick=2, fmt='s-', label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]))
    ax.fill_between(df["theta_val"], df[f"{metric_name}_avg"] - df[f"{metric_name}_std"], df[f"{metric_name}_avg"] + df[f"{metric_name}_std"], alpha=0.2)
ax.legend()
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("PSNR")
ax.set_ylim(0.65, 0.9)
plt.show()

In [ ]:
metric_name = "wssb_mean_test_ssim_gt"
fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
for i in range(len(color_vec)):
    sigma = sigma_vec[i]
    df = new_df[new_df["sigmaRui_sq"] == sigma]
    ax.plot(df["theta_val"], df[f"{metric_name}_avg"], 's-', lw=2, label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i])
ax.legend()
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("SSIM")
ax.set_ylim(0.65, 0.9)
plt.show()

In [ ]:
metric_name = "wssb_mean_test_ssim_gt"

fig, ax = plt.subplots(figsize=FIGSIZE)
color_vec = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
sigma_vec = new_df["sigmaRui_sq"].unique()
width = 0.2
ind = np.arange(len(new_df["theta_val"].unique()))
for i in range(len(color_vec)):
    df = new_df[new_df["sigmaRui_sq"] == sigma_vec[i]]
    ax.bar(x = ind + i*width, height = df[f"{metric_name}_avg"], width = width, label=r"$\gamma^{\mathrm{Rui}}=" + r"{}$".format(sigma_vec[i]), color=color_vec[i], edgecolor='black', yerr=df[f"{metric_name}_std"], capsize=3)
ax.set_xticks(ind+(3*width)/2, new_df["theta_val"].unique())
ax.legend(loc='best')
ax.set_xlabel(r"$\theta$")
ax.set_ylabel("SSIM")
ax.set_ylim(0.65, 0.98)
plt.savefig(SAVE_PATH + "ssim_bar.pdf", bbox_inches='tight')
plt.show()

In [ ]:
#df = pd.read_csv(history_file)
#df.head()

In [ ]:
def plot_metric(df, metric, title=None):
    plt.plot(df['train_' + metric], label='train')
    plt.plot(df['val_' + metric], label='validation')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    if title is not None:
        plt.title(title)